In [1]:

import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm


# def generate_cir(N):
#     x=np.zeros([N,N])
#     for i in range(1,N-1):
#         x[i][i]=1/3
#         x[i][i+1]=1/3
#         x[i][i-1]=1/3
#     x[N-1][N-2]=1/3
#     x[N-1][N-1]=1/3
#     x[N-1][0]=1/3
#     x[0][N-1]=1/3
#     x[0][0]=1/3
#     x[0][1]=1/3
#     return x

def generate_ful(N):
    x=np.zeros([N,N])
    for i in range(N):
        for j in range(N):
            x[i][j]=1/N
    return x

def generate_compete(N, delta):
    x = np.identity(N)
    A = np.ones([N,N]) - x
    degree = [N-1] * N
    D = np.diag(degree)
    w = D - A
    x = x- delta*w
    return x
    

# def generate_star(N):
#     x = np.zeros([N,N])
#     for i in range(N):
#         if i ==0 :
#             for j in range(N):
#                 x[i][j] = 1/N
#         else:
#             x[i][0] = 1/8
#             x[i][i] = 7/8
#     return x

def generate_star_new(N, delta):
    x = np.identity(N)
    A = np.zeros([N,N])
    for i in range(N-1):
        j = i + 1
        A[0,j] = 1
        A[j,0] = 1
    degree = [N-1] + [1] *(N-1)
    D = np.diag(degree)
    w = D - A
    x = x - delta * w
    return x

def alpha_estimator(m, X):
    # X is N by d matrix
    N = len(X)
    n = int(N/m) # must be an integer
    Y = np.sum(X.reshape((n, m, -1)), 1)
    eps = np.spacing(1)
    Y_log_norm = np.log(np.linalg.norm(Y, axis=1) + eps).mean()
    X_log_norm = np.log(np.linalg.norm(X, axis=1) + eps).mean()
    diff = (Y_log_norm - X_log_norm) / np.log(m)
    return 1/diff

# a_{i,j}: \sim N (0, std_a^2 I_d)
#  y = <a^T, x> + noise
# noise \sim N (0, std_noise^2)
def generate_data(std_a, std_noise, d, batch):
    data_a = []
    data_y = []
    for i in range(batch):
        a = np.random.normal(0, std_a, d)
        noise = np.random.normal(0, std_noise, 1)
        y = np.dot(a, x_true) + noise
        data_a.append(a)
        data_y.append(y)
    data_a = np.array(data_a)
    data_y = np.array(data_y)
    return data_a, data_y

def MSE(x, data_a, data_y):
    batch = len(data_y)
    mse = 0
    for i in range(batch):
        diff = np.dot(data_a[i], x) - data_y[i]
        mse += diff**2
    return mse / (2*batch)

def gradient(x, data_a, data_y):
    batch = len(data_y)
    grad = 0
    for i in range(batch):
        diff = np.dot(data_a[i], x) - data_y[i]
        diff = diff / batch * data_a[i]
        grad += diff
    return grad

In [9]:
nodes = 1
d = 1
x_true = np.random.normal(0,3,d)
X_true = np.ones((nodes,1)) @ x_true.reshape(1,-1)
batch = 10
iterations = 1000
num_last_iters = 500
std_nodes = [1] * nodes
std_noise = 0.2
num_exp = 1600
# initialization 
x0 = np.random.uniform(-3,3,d)
lr_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
delta_list = [0, 0.025, 0.05]


In [10]:
final_iter_delta = []
alpha_all_delta = []
error_iter_delta = []
for delta in delta_list:
    print(delta)
    W = generate_compete(nodes, delta)
#     print('star start!')
    
    final_iter_all = []
    error_iter_all = []
    for lr in lr_list:
        print(lr)
        iteration = int(iterations/lr)
        Xd_across_exp = np.zeros((nodes, num_exp, d))
        error_record_d_averaged = np.zeros(iteration)
    
        for e in range(num_exp):
    
            #if e%100 == 0:
            #    print(e)
    
            Xd = np.ones((nodes,1)) @ x0.reshape(1,-1) # each row i is x[i]
            error_record_d = np.zeros(iteration)
            Xd_averaged = np.zeros((nodes, d))
    
            for ite in range(iteration):
    
                # sample data and get gradient
                G = np.zeros((nodes, d)) # each row i is gradient of node i
                for i in range(nodes):
    
                    # sample data
                    A = np.random.randn(batch, d)
                    noise = np.random.randn(batch,1) * std_noise
                    b = A @ x_true.reshape(-1,1) + noise
    
                    # compute gradient
                    g = (1/batch) * (A.T @ (A @ Xd[i,:].reshape(-1,1) - b))
                    G[i,:] = g.reshape(-1)
    
                # main update
                Xd = W @ Xd - lr * G
                
                # average the last 1000 iterates
                if ite >= iteration - num_last_iters:
                    Xd_averaged += Xd
    
                # compute distance to the true solution
                error = np.linalg.norm(Xd - X_true,'fro')
                error_record_d[ite] = error
                
            # fill in Xd_across_exp 
            Xd_averaged /= num_last_iters
            
            for i in range(nodes):
                Xd_across_exp[i,e] = Xd_averaged[i]
                
            error_record_d_averaged += error_record_d
            
        error_record_d_averaged = error_record_d_averaged/num_exp
        final_iter_all.append(Xd_across_exp)
        error_iter_all.append(error_record_d_averaged)
#     final_iter_delta.append(final_iter_all)
    error_iter_delta.append(error_iter_all)
    
    #np.save('final_iter_all_star.npy',final_iter_all)
    #np.save('error_iter_star.npy', error_iter_all)
    
    print('star finished!')
    
    alpha_nodes_all = []
    #alpha_cent_all = []
    for l in range(len(final_iter_all)):
        alpha_nodes = []
        for i in range(nodes):
            data_alpha = np.array(final_iter_all[l][i]-np.mean(final_iter_all[l][i],axis=0))
            alpha_nodes.append(alpha_estimator(40, data_alpha))
    
        #data_alpha = np.array(final_iter_cent_all[l]-np.mean(final_iter_cent_all[l], axis=0))
        #alpha_cent = alpha_estimator(40, data_alpha)
        alpha_nodes_all.append(alpha_nodes)
        #alpha_cent_all.append(alpha_cent)
    alpha_nodes_all = np.transpose(alpha_nodes_all)
    alpha_all_delta.append(alpha_nodes_all)
    final_iter_delta.append(final_iter_all)

0
star start!
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1
star finished!


In [11]:
for alpha_delta in alpha_all_delta:
    print(np.median(alpha_delta,axis=0))

[1.75889073 2.31563775 2.04913005 1.96998153 1.79766576 2.08269534
 1.84714428 1.92713215 2.04364166 1.88575881]
